In [ ]:
!python bio-medrxiv.py testeins -f /data/Goldlabel_biomedRxiv/goldlabel1_docs801-840_laura/goldlabel1_tasknameshort_MedRxiv_docs/ -c -M -C -L

In [1]:
!python bio-medrxiv.py testeins -L

[2023-06-22 00:01:21,207][INFO] fonduer.meta:49 - Setting logging directory to: /tmp/2023-06-22_00-01-21
[2023-06-22 00:01:21,346][INFO] fonduer.supervision.labeler:358 - Clearing ALL Labels and LabelKeys.
/usr/local/lib/python3.8/site-packages/fonduer/utils/utils_udf.py:217: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  .filter(candidate_class.id.in_(sub_query))
[2023-06-22 00:01:21,667][INFO] fonduer.supervision.labeler:330 - Clearing Labels (split 0)
/usr/local/lib/python3.8/site-packages/fonduer/supervision/labeler.py:340: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  query = self.session.query(table).filter(table.candidate_id.in_(sub_query))
[2023-06-22 00:01:21,671][INFO] fonduer.utils.udf:67 - Running UDF...
100%|███████████████████████████████████████████| 14/14 [00:04<00:00,  3.35it/s]
[2023-06-22 00:01:26,195][INFO] fonduer.supervision.labe

imports

In [2]:
from fonduer.supervision import Labeler
from fonduer.features import Featurizer
from fonduer.candidates.models import Candidate

from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel

from MeMoKBC.pipeline.utils import get_session, load_candidates, match_label_matrix
from MeMoKBC.definitions.candidates import NameFullAbbr, NameAbbrTask
from MeMoKBC.pipeline.lfs.name_short_long_lfs import short_long_lfs
from MeMoKBC.pipeline.lfs.name_short_task_lfs import name_abbr_task_lfs
from MeMoKBC.gold_label_matcher import match_gold_label
from importlib import reload
import csv

Get session object

In [3]:
session = get_session(db_name="testeins")

[2023-06-22 00:01:50,496][INFO] fonduer.meta:49 - Setting logging directory to: /tmp/2023-06-22_00-01-50


In [4]:
FALSE = 0
TRUE = 1
ABSTAIN = -1

def is_medical_abbreviation(c):
    name_abbr, task = c
    name_span = name_abbr.context.get_span()

    with open('/workspaces/bio-medRxiv/data/CSVs/all_abbs.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        medical_abbreviations = [row[0].lower() for row in reader]

    for ngram in name_span:
        if ngram.lower() in medical_abbreviations:
            return FALSE

    return ABSTAIN

In [5]:
cands = session.query(NameAbbrTask).all()

In [19]:
cands[10]

NameAbbrTask(NameAbbr(SpanMention("GWAS", sentence=46890, chars=[6,9], words=[2,2])), Task(SpanMention("Potentially, our method could profit from better ways to deal with GWAS pairs whose traits exhibit very different effect size distributions or sample sizes.", sentence=60550, chars=[0,155], words=[0,25])))

In [28]:
for cand in cands:
    output = is_medical_abbreviation(cand)
    print(output)

1
1
1
1
1
1
-1
1
1
-1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
-1
-1
-1
1
1
-1
1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
-1
-1
1
1
1
-1
-1
-1
-1
-1
-1
1
1
1
-1
-1
1
-1
-1
1
1
1
1
1
-1
-1
-1
-1
1
1
1
-1
1
1
-1
-1
-1
-1
1
1
1
1
1
-1
-1
1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
1
1
1
-1
-1
-1
1
1
1
1
1
1
1
1
-1
-1
-1
1
1
1
1
1
1
1
1
-1
-1
-1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


Define candidates and Labeler object

In [4]:
candidates = [NameAbbrTask, NameFullAbbr]

Load candidates and labels

In [5]:
L_train_NAT, L_train_NFA = match_label_matrix(session, candidates, 0)

/workspaces/bio-medRxiv/MeMoKBC/src/MeMoKBC/pipeline/utils.py:55: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  cands = (session.query(candidate_class).filter(candidate_class.id.in_(sub_query)).order_by(candidate_class.id).all())


## LF analysis

NameFull + Abrreviation

In [6]:
LFAnalysis(
    L_train_NFA,
    lfs=sorted(short_long_lfs, key=lambda lf: lf.name)
).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check_all_uppercase_letters,0,[],0.000000,0.000000,0.0
name_full_in_top_percentile_sentence_wise,1,[1],0.267387,0.036739,0.0
name_short_outside_half_percentile_sentence_wise,2,[],0.000000,0.000000,0.0
small_letter_count,3,[1],0.069745,0.052063,0.0
word_count,4,[1],0.098232,0.054224,0.0


NameAbbr + Task

In [7]:
LFAnalysis(
    L_train_NAT,
    lfs=sorted(name_abbr_task_lfs, key=lambda lf: lf.name)
).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
is_medical_abbreviation,0,[],0.000000,0.000000,0.0
lf_length_more_than_three_words,1,[0],0.891705,0.013825,0.0
lf_name_short_in_first_words,2,[0],0.020737,0.013825,0.0
